In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from IPython.display import HTML
from IPython.display import Javascript

# https://docs.opencv.org/trunk/da/d54/group__imgproc__transform.html

# https://docs.opencv.org/trunk/d4/d35/samples_2cpp_2polar_transforms_8cpp-example.html#a16

# https://python-forum.io/Thread-Image-conversion-form-cartesian-to-polar-and-back-to-cartesian

#https://en.wikibooks.org/wiki/LaTeX/Mathematics

# НАДЕЖНАЯ РЕГИСТРАЦИЯ ИЗОБРАЖЕНИЙ С ИСПОЛЬЗОВАНИЕМ ЛОГ-ПОЛЯРНОГО ПРЕОБРАЗОВАНИЯ

In [2]:
src = cv.imread('/Users/kolsha/Pictures/i_m_pumpkin_rick__stencil_by_batnamz-dbrxwip.png')
src = cv.cvtColor(src, cv.COLOR_BGR2RGB)

rows, cols, ch = src.shape
crow, ccol = int(rows/2) , int(cols/2)

@interact_manual(rotate=(-ccol, ccol), scale=(-crow, crow))
def show_log_polar(rotate, scale):
         # center
    M = 121.0
    
    rotate *=4
    scale *=2

    #polar_img = cv.logPolar(src, (crow, ccol), M)

    img64_float = np.copy(src)#.astype(np.float64)

    center = (ccol, crow)

    #center = (0, 0)

    #M = np.sqrt(((img64_float.shape[0]/2.0)**2.0)+((img64_float.shape[1]/2.0)**2.0)) / 4
    print(M, src.shape)

    #log_polar_img = cv.logPolar(img64_float, center ,M ,cv.INTER_LINEAR + cv.WARP_FILL_OUTLIERS)
    #cv.linearPolar(img64_float,(img64_float.shape[0]/2, img64_float.shape[1]/2),Mvalue,cv.WARP_FILL_OUTLIERS)

    #cartisian_image = cv.linearPolar(ploar_image, (img64_float.shape[0]/2, img64_float.shape[1]/2),Mvalue, cv.WARP_INVERSE_MAP)
    flags = cv.INTER_LINEAR + cv.WARP_FILL_OUTLIERS + cv.WARP_POLAR_LOG

    maxRadius = 1.5 * np.max(center)
    #print(maxRadius)
    log_polar_img = cv.warpPolar(img64_float, (0,0) ,center, maxRadius, flags)


    #log_polar_img = log_polar_img/255

    log_polar_img = np.roll(log_polar_img, (rotate, scale), (0, 1))

    #cartisian_image  = cv.logPolar(log_polar_img, center, M, cv.WARP_INVERSE_MAP + cv.INTER_LINEAR + cv.WARP_FILL_OUTLIERS)


    cartisian_image = cv.warpPolar(log_polar_img, (cols,rows) ,center, maxRadius, flags + cv.WARP_INVERSE_MAP)
    
    
    titles = ['Original Image','Log Polar','Cartisian']
    images = [src, log_polar_img, cartisian_image]
    fig = plt.gcf()
    fig.set_size_inches(20, 15.5)
    for i in range(len(images)):
        plt.subplot(2, 3, i+1),
        plt.imshow(images[i], aspect='auto')
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()

interactive(children=(IntSlider(value=0, description='rotate', max=400, min=-400), IntSlider(value=0, descript…

# Полярные координаты


## $r = \sqrt{(x-x_c)^2 + (y - y_c)^2}$


## $\varphi = \tan^{-1} (\frac{y - y_c}{x-x_c})$

# Алгоритм:

0. ## Вырезаем центральный регион $I^{'}_1$ из $I_1$

0. ## Переводим в лог-полярную ситему $I^{'}_1$ -> $I^{'}_1p$

0. ## Для всех позиций $(x,y)$ из второго изображения $(I_2)$: 
    0. ## Вырезаем регион $I^{'}_2$ из $I_2$
    0. ## Переводим в лог-полярную ситему $I^{'}_2$ -> $I^{'}_2p$
    
    0. ## Считаем кросс-корреляцию $I^{'}_1p$ и $I^{'}_2p$ -> $(dx,dy)$
    
    0. ## Если корреляция максимальна, сохраняем $(x,y)$ и $(dx,dy)$

0. ## Масштабируем на $(dx)$:
0. ## Вращаем на $(dy)$:
0. ## Сдвигаем в $(x,y)$

0. # You're great

# Пирамидный подход к субпиксельной регистрации на основе интенсивности

# Введение
<br/>

0. ### Используют неизмененную интенсивность всех пикселей изображения



0. ### Используют более высокий порядок интерполяции для минимизации размытия изображения и достижения согласованности в вычислении пространственных производных



0. ### Их модель деформации состоит из комбинации полного 3-D аффинного преобразования и дополнительного линейного изменения контраста. Они получают более простые модели, ограничивая аффинное преобразование конкретными подмножествами параметров, реализуя комбинации вращения, трансляции и изометрического масштабирования.


0. ### Нелинейный алгоритм оптимизации Марквардта-Левенберга:
    0. #### Они ускоряют его выполнение, используя особую структуру своей модели деформации.
    0. #### В частности, они переформулируют задачу оптимизации таким образом, чтобы они могли предварительно вычислить большинство терминов, необходимых для построения Гессиана и градиентного критерия, вместо того, чтобы переоценивать их на каждой итерации, как это требуется в традиционном подходе.
    0. #### Они включили Марквардта-Левенберга в мультиразрешающую структуру, используя стратегию итерации от грубого до тонкого и распространяя оценки для одного уровня пирамиды разрешения от ее предыдущего уровня.

# Критерий подобия тестовых данных эталонным

## $\varepsilon^2 = \iint_{\{x\} \subset R^{q}} \mathrm{(f_R(x) - Q_p\{f_T(x)\})^2}\,\mathrm{d}x $

### $ f_R - \text{исходные данные} \\ f_T - \text{тестовые данные} \\ Q_p\{f\} - \text{преобразование осуществляющееся по } \textbf{p} \\ q - \text{размерность пространства} $

### $ \text{Такой критерий хорошо поддается минимизации по отношению к }\textbf{p}\text{ и хорошо понятен.} $